# Extracción de datos en Python


Para comenzar nosotros trabajamos con nuestros datos para el TFM y en este caso en Python vamos a hacer la extracción, una pequeña limpieza y su tranformación a fichero. 

En los anexos tenemos los intentos de extracción erroneos. 

## Creamos las funciones 

#### Función que abre el archivo

In [1]:
def abreArchivo(nombre):
    try: 
        fichero = (open(str(nombre)+'.txt', 'x')) 
        print('se ha CREADO el archivo') 
        fichero.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Archivo ya existe') 
    try: 
        fichero = (open(str(nombre)+'.txt', 'a')) 
        print('Archivo abierto en modo APPEND') 
    except Exception as e:
        print('Se ha producido un Error al intentar ABRIR el archivo en modo APPEND\n', e)
    return fichero

Función que crea un archivo con un parametro str que le da nombre y que deja el archivo abierto en modo Append.

Try revisa que exista el archivo y sino lo crea.

Intentamos crear el archivo y nos dá un mensaje de exito o si existe uno de error.

Tras esto, lo abre en modo append y en caso positivo nos devuelve de nuevo el mensaje de exito, en caso de error nos devolverá el mensaje correspondiente 

Finalmente nos devuelve el fichero.



#### Función que cierra archivo 

In [2]:
def cierraArchivo(nombre):
    try:
        nombre.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Error')

Pasa la variable entera para que la cierre o de mensaje de error

#### Función que saca atributos 

In [3]:
def saca_atributos(items):
    x = len(items)-1
    lista = []
    while x >= 0:
        lista.append(items[x]['attributes'])
        x = x - 1
    return lista

Esta función extrae los atributos mientras el item sea menor o igual a 0 

Creamos un bucle que recorra todos los datos y creamos una lista vacia para introducir los datos solicitados en ella.

Mientras nuestra variable(objeto) sea cierta añadirá a la lista los datos extraidos. 

Imprimimos la lista para mostrarla

#### Función que saca los datos geométricos y los inserta con el Object ID

In [4]:
def saca_geografia(items):
    x = len(items)-1
    diccionario = {}
    while x >= 0:
        a = items[x]['attributes']
        b = items[x]['geometry']
        diccionario['objectID'] = a['OBJECTID'] 
        diccionario['rings'] = b['rings']
        x = x - 1
    return diccionario

Esta función saca la geografía creando un diccionario vacío e introduciendo los datos que estan dentro de 'rings'.

A continuación nos devuelve los datos extraidos en un diccionario.

### Importamos requests para hacer las llamadas

In [5]:
import requests

### Variables para bucles y llamadas

In [6]:
i = 0
limite = 2
offset = 1
count = 1

Las dos primeras variables son para controlar el while (controlan numero de veces que se repite el bucle)
Las siguientes dos variables son para controlar el URL (controlan que pido en cada request)

### Hacemos la llamada para extraer los datos

#### Primero damos valor a cada parte de la URL

In [7]:
inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' 
where = '&where=(1%3D1)%20AND%20(1%3D1)' 
geometry = '&returnGeometry=true'
spatial = '&spatialRel=esriSpatialRelIntersects' 
outfields = '&outFields=*' 
orderby = '&orderByFields=OBJECTID%20ASC'
outSR = '&outSR=102100' 
resultOffset = '&resultOffset=1' 
resultRecord = '&resultRecordCount=2' 

###### Explicación de las equivalencias
f = Formato en el que pasara los datos

where = Condicion para la consulta

geomtry = Si pasa la geometria o no

orderby = El orden de la consulta

resultOffset = Cuantas columnas desde el principio antes de empezar a imprimir

resultRecord = Cuantas columnas imprimir






#### Cantidad de columnas de la llamada

resultOffset = '&resultOffset=1'

El número 1 indica el comienzo de la consulta

resultRecord = '&resultRecordCount=2'

El número 2 indica la llamada a el número de columnas que escojamos

### Generamos el archivo

In [8]:
BD_objetos = abreArchivo('BD_objetos')
BD_atributos = abreArchivo('BD_atributos')
BD_objGeograf = abreArchivo('BD_objGeograf')

Archivo ya existe
Archivo abierto en modo APPEND
Archivo ya existe
Archivo abierto en modo APPEND
Archivo ya existe
Archivo abierto en modo APPEND


Generamos los 3 ficheros que necesitamos con objetos, atributos y Geografía 

### Extracción a fichero

In [9]:
while i < limite:
    url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+str(offset)+resultRecord+str(count)
    res = requests.get(url)
    data = res.json()
    items = data['features']
    
    BD_objetos.write(str(items)) 

    atributos = saca_atributos(items)
    BD_atributos.write(str(atributos))
                     
    geografia = saca_geografia(items)
    BD_objGeograf.write(str(geografia))
                     
    print(offset) 
    offset += 1 
    i += 1 

1
2


Creamos un bucle que itere por la cantidad de archivos que hayamos especificado
1. Unimos todos los valores en una URL
2. Hacemos una llamada en formato JSON y pedimos los datos
3. Solicitamos los datos dentro de 'features'
4. Escribimos los datos en el archivo en formato string
5. Utilizamos la función que saca los atributos y los escribe igual que el anterior
6. Usamos la función geografía y hará el mismo proceso
7. Imprimimos un contador de control para log 
8. Aumentamos el offset para pasar al proximo item
9. Finalmente aumentamos el control del bucle

### Cerramos los archivos

In [10]:
cierraArchivo(BD_objetos)
cierraArchivo(BD_atributos)
cierraArchivo(BD_objGeograf)

Se ha CERRADO el archivo
Se ha CERRADO el archivo
Se ha CERRADO el archivo


# Seguidamente, vamos a usar la información geográfica de cada litología para representar los polígonos correspondientes sobre un mapa.

In [1]:
### En primer lugar, vamos a instalar las librerías necesarias. 
### folium para representar el mapa, y proj para transformar las coordenadas.

import folium
from pyproj import Proj, transform

### Llamamos a la API y obtenemos toda la info sobre un objeto de nuestra DB

In [2]:
# SACO DATOS

import requests

inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' #Formato en el que pasara los datos
where = '&where=(1%3D1)%20AND%20(1%3D1)' #condicion para la consulta
geometry = '&returnGeometry=true'#si pasa la geometria o no
spatial = '&spatialRel=esriSpatialRelIntersects' #AVERIGUAR
outfields = '&outFields=*' #AVERIGUAR
orderby = '&orderByFields=OBJECTID%20ASC' #como viene ordenada la consulta
outSR = '&outSR=102100' #AVERIGUAR
resultOffset = '&resultOffset=250500' #cuantas columnas desde el principio antes de empezar a imprimir
resultRecord = '&resultRecordCount=1' #cuantas columnas imprimir

url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+resultRecord
    
res = requests.get(url)
data = res.json()
item = data['features']
item

[{'attributes': {'HOJA': '442',
   'ID': 25,
   'DLO': 'Arcillas y limos. Zona endorreica',
   'OBJECTID': 250501},
  'geometry': {'rings': [[[-16002.505861943131, 5047016.888726178],
     [-15998.433238368883, 5047001.752278435],
     [-15988.285353593285, 5046972.963576308],
     [-15983.200167932433, 5046944.170815077],
     [-15983.185473756852, 5046925.537646988],
     [-15981.48451194233, 5046908.59971282],
     [-15969.658708477242, 5046891.670097758],
     [-15952.772988874243, 5046878.1323776785],
     [-15922.374084965099, 5046847.6664707055],
     [-15905.472669321696, 5046813.801858466],
     [-15891.943677644176, 5046776.546866025],
     [-15881.796349460152, 5046747.758833892],
     [-15869.977336488128, 5046739.29882611],
     [-15861.539987003258, 5046739.305487177],
     [-15846.35667637184, 5046744.399431462],
     [-15834.558702782548, 5046763.041724294],
     [-15799.150421787703, 5046800.335743271],
     [-15765.404586072234, 5046805.444226114],
     [-15719.842409

### Podemos ver que la información obtenida (que anteriormente hemos reunido en los archivos finales BD_objetos, BD_atributos y BD_objGeograf) contiene 5 variables:
#### Hoja : El número de hoja en la clasificación MAGNA en la que se encuentra el objeto requerido
#### DLO : Descripción litológica. Descripción de la unidad rocosa correspondiente al objeto en cuestión
#### ObjectID: Identificador de tipo de objeto dentro de la API
#### ID: Identificador de DLO
#### geometry: siguiendo un modelo ESRI, cada objeto tiene un 'ring' asociado. Cada ring se compone de una lista indefinida de puntos que, unidos entre sí, marcan la limitación geográfica del objeto solicitado.

### A continuación, limpiamos los datos para poder trabajar con las geometrías en solitario

In [3]:
# SEPARO LA LISTA DE RINGS

rings = item[0]['geometry']
target = rings['rings']
target

[[[-16002.505861943131, 5047016.888726178],
  [-15998.433238368883, 5047001.752278435],
  [-15988.285353593285, 5046972.963576308],
  [-15983.200167932433, 5046944.170815077],
  [-15983.185473756852, 5046925.537646988],
  [-15981.48451194233, 5046908.59971282],
  [-15969.658708477242, 5046891.670097758],
  [-15952.772988874243, 5046878.1323776785],
  [-15922.374084965099, 5046847.6664707055],
  [-15905.472669321696, 5046813.801858466],
  [-15891.943677644176, 5046776.546866025],
  [-15881.796349460152, 5046747.758833892],
  [-15869.977336488128, 5046739.29882611],
  [-15861.539987003258, 5046739.305487177],
  [-15846.35667637184, 5046744.399431462],
  [-15834.558702782548, 5046763.041724294],
  [-15799.150421787703, 5046800.335743271],
  [-15765.404586072234, 5046805.444226114],
  [-15719.842409043255, 5046805.480048119],
  [-15670.900127002491, 5046798.742849273],
  [-15644.157289127845, 5046800.911411473],
  [-15653.026112958341, 5046789.775812751],
  [-15673.008740796757, 5046770.84

# Seguidamente, transformamos las coordenadas en un formato manejable. La API nos las suministra en pseudo-mercator, y las transformamos al modelo (latitud , longitud) que pueda leer folium.

In [4]:
# RECORRO LA LISTA Y TRANSFORMO EL FORMATO DE LAS COORDENADAS

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
lista = target[0]
convertidas = []
a = 0

while a < len(lista):
    x1,y1 = lista[a]
    punto = transform(inProj,outProj,x1,y1)
    convertidas.append(punto)
    a += 1

convertidas

[(-0.1437529559999575, 41.23466723206472),
 (-0.14371637099992537, 41.234564978064704),
 (-0.14362521099997133, 41.23437049606474),
 (-0.14357952999995405, 41.23417598606477),
 (-0.14357939799992894, 41.23405010906475),
 (-0.14356411799997204, 41.23393568406476),
 (-0.14345788499997522, 41.23382131506473),
 (-0.14330619799994793, 41.23372986006473),
 (-0.14303311999992727, 41.23352404506476),
 (-0.14288129199997002, 41.233295269064726),
 (-0.14275975899994361, 41.23304358806473),
 (-0.14266860399993675, 41.23284910606474),
 (-0.14256243199997698, 41.232791953064755),
 (-0.14248663799997982, 41.232791998064734),
 (-0.14235024399994245, 41.232826411064764),
 (-0.14224426099997345, 41.23295235206473),
 (-0.14192618299995274, 41.23320429706472),
 (-0.14162303899996684, 41.233238808064755),
 (-0.1412137469999379, 41.23323905006476),
 (-0.14077409099996888, 41.23319353606473),
 (-0.14053385599993362, 41.233208186064765),
 (-0.1406135259999246, 41.23313295806472),
 (-0.14079303299996582, 41.2

## Además, invertimos los datos para que las coordenadas tengan coherencia

In [5]:
#INVERTIR TUPLAS

b = 0
coordenadas = []

while b < len(convertidas):
    punto1 = convertidas[b][1]
    punto2 = convertidas[b][0]
    entrar = (punto1, punto2)
    coordenadas.append(entrar)
    b += 1

coordenadas

[(41.23466723206472, -0.1437529559999575),
 (41.234564978064704, -0.14371637099992537),
 (41.23437049606474, -0.14362521099997133),
 (41.23417598606477, -0.14357952999995405),
 (41.23405010906475, -0.14357939799992894),
 (41.23393568406476, -0.14356411799997204),
 (41.23382131506473, -0.14345788499997522),
 (41.23372986006473, -0.14330619799994793),
 (41.23352404506476, -0.14303311999992727),
 (41.233295269064726, -0.14288129199997002),
 (41.23304358806473, -0.14275975899994361),
 (41.23284910606474, -0.14266860399993675),
 (41.232791953064755, -0.14256243199997698),
 (41.232791998064734, -0.14248663799997982),
 (41.232826411064764, -0.14235024399994245),
 (41.23295235206473, -0.14224426099997345),
 (41.23320429706472, -0.14192618299995274),
 (41.233238808064755, -0.14162303899996684),
 (41.23323905006476, -0.1412137469999379),
 (41.23319353606473, -0.14077409099996888),
 (41.233208186064765, -0.14053385599993362),
 (41.23313295806472, -0.1406135259999246),
 (41.23300507306476, -0.1407

## Finalmente, mandamos la lista de coordenadas a folium para poder representar (en rojo) el polígono correspondiente al objeto solicitado. Añadimos a mano un popup descriptivo de la zona en cuestión (más adelante lo relacionaremos directamente con la DLO).

In [6]:
#PRUEBA DE DIBUJO
m = folium.Map(location = coordenadas[0], zoom_start=16)

folium.Polygon(coordenadas, color="red", weight=2.5, fill_color='red', opacity=10, popup = 'Arcillas y limos. Zona endorreica').add_to(m)

m

## Anexos

### Intento con WebScraping

#### Importamos los paquetes

In [11]:
import requests
from bs4 import BeautifulSoup

#### Creamos un bucle para extraer la información de los PDF online

In [12]:
for numero in [1,2]:
    URL = 'http://info.igme.es/cartografiadigital/datos/magna50/pdfs/d0_G50/Magna50_'+str(numero)+'.pdf'
    page = requests.get(URL)
    #page.enconding()
    soup = BeautifulSoup(page.content,'html.parser')#.decode('utf-8','WinAnsiEncoding')
    print(page.content)
    
soup = BeautifulSoup(page.content,'html.parser')
hojas = soup.find_all('td', class_='EnlaceMini')
hojas

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


b'%PDF-1.6\r\n1 0 obj\r\n<<\r\n/Length 2 0 R\r\n/Filter  /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xec\xbd[\xb3d\xcbq\x1e\xf6\x8e\x08\xfc\x87\x1dz\xb0iE\xb0U\xf7\x8b#\xfc@\x02\x10\r\x99 %\x9cc\xfb\xc1p( \x00\x84A\xcf\x00\x14H\x8a\xa1\x7f\xef\xfc\xbe\\\xdd]k\xed\xca\xde\xbd\xbbk\x9f90\xe7\x80\x00g\xf2\xac\xce\xbadU\xde*/\xee\xc5\xc9\x7f~\xfeW?\xfc\x81\x7f\xc1\x7f\xfe\xf8\xdb\x1f\xfe\xe0\xdf\xfd\xed\x8f^\xe4\xbf\xee\xe5/\x7f\xfc\xa3\xcb\xdf\xbc\xfe\xed\xbf\xfe\xf0\x07\xc5\xe5|\n/\xee\xe4b,/\x9f\x81@\xff\xf8I\xfe\x98B-\xa7&\x7f\xdc\xbe:\xfe\xfd\xf2\xe9\xff\xf3\xc3\x1f\xfc\x9f\xff\xf6\xe5\xf7\xc4\x98\\+\xa7\x1a\xbb\xab\x9c\xcd\x9f\x97r\xea\xb9\xa7\xfa\xe2c\x0c\'\xe7J\x08/\xb1\xe7pj.\xe4\xfc\xf2\xab\xcf2\xab\x9f~\xfe\xe5o\x7f\xf3\x9f\xe3\xcb\x8f\xff\xf0\xc3\x1f\xfc\xa7\xfb\xb14\xf9&\x97(\xdf\x0cX\xf2\xbb\xb1\x04w\x8a!\xe6>b\xa9\xef\xc5Rs:\xb9\x96\xbc\x1f\xb1\xf4\xf7b)\xad\x9eZJ%\x8eX\x04\xe7{\xd1\x04\x7f\x12\x1a\xf9\xdd\xf6\xfaw\xefo\x16*\xc7\x92\xcbn\x7f\xfd\xbb78\xb5~\xf2\xa1\xf8\xdd\x06\xfbw

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


b'%PDF-1.5\r\n1 0 obj\r\n<<\r\n/Length 2 0 R\r\n/Filter  /FlateDecode\r\n>>\r\nstream\r\nx\x9c\xec\xbdi\xd3\xa2\xda\xb65\xf8\xdd\x08\xffC\xd2\x88(\x8d "\xa0(6\x80}\xdf\x8b\xa0\xa8\xa8\xd8\xa1\xa8U\xf1\xfa\xa1~{\x81\xfad\xe6\xbew\xef{\x9f\xb3c\xc7\x89\x8a\xa8\x93\x19\x99\xf9(\xb8\x9a9\xe6\x9cc\xcc\xb5\x16&\xf5\x83\xf2~wK\xc1\x00\xfd\xc3\xff\xedn\x83\x81X\xdd\xfc?\x96\xfbC\x14c}\xfb~\xb4~\xa0\x95\x93\xb9\xb5"\xb1\xa1}\xb3\x97\xde\xeb\xbb\xfb\xb0~\xc4\xda\xae\xf5\x7f\xd9\xd6\xff\xfdy\xa5\xac\xed\xbb\xf9\xfbE\xfb|\xb7\xd6\x9fW\xb2}:y/6\xe6\xf1\xe6\xbd*:G\xc7\xfd\xa1\xc5\xa9x<\xf1\x83\x89sI\xfeG\x92e\x19\xf6\x87\x1e\xdb\x1c\xcd\xfb\xbb\xf3\xcf\xbd}\xd7<\xdf.\xa6k\x9dW\xff\xe7\xf3^6[\x90\x8b\xde [\xc5\x1f\xde\x1f\xea\xc7\xeb\xd55\x18\xf0\xe7q\xf2\xfe\xd0I\x9e!9\x96\xa6\xf9\x1f\xc7\x1fq\x86O\x90\xf1\x04\xcf\xc7\xff\xf2}\xca{I\xbd\xfe\xde\x05\x03\xa3\xe8\x8f\xf3\xab5:I\xf1$\x1f\xa78\xeeu\x91H\xc4\xc9\x04\'\xf0?\xd8x\x82di\x86c~\xd0,\x9b$\x19\x9a\xe6\xf8\x1f\xabS0P\xa8\x04\x03?~\xc4F?\x12\x89\x

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[]

Este bucle itera entre los archivos 1 y 2 puesto que es de prueba.

La URL esta concatenada con la variable para los distintos archivos.

Utilizamos el get para pedir la información y BeautifulSoup para hacer un parser para transformar el contenido y tras esto imprimirlo. 

En este caso tenemos comentado las dos formas que intentamos para decodificar el archivo.

No pudimos continuar porque extraiamos los datos pero nos surgía un problema con el que no podiamos avanzar, el archivo tenía dos codificaciones distintas UTF-8 y WinAnsiEncoding. 

Por ello, conseguimos sacar la información con Webscraping pero no como la necesitabamos.



### Intento con librerias Python 

In [13]:
import PyPDF2
pdf_file = open('Magna50_1.pdf')
read_pdf = PyPDF2.PdfFileReader(pdf_file)
number_of_pages = read_pdf.getNumPages()
page = read_pdf.getPage(0)
page_content = page.extractText()
print (page_content)

UnsupportedOperation: can't do nonzero end-relative seeks

Intentamos de forma erronea utilizar la libreria PyPDF2

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice

fp = open('Magna50_1.pdf', 'rb')
# Creamos un parse de PDF asociado al PDF
parser = PDFParser(fp)
# Creamos un objeto PDF con la estructura
# Damos contraseña para iniciar
document = PDFDocument(parser)
# Chequeamos si el documento permite la extracción
if document.is_extractable:
    raisePDFTextExtractionNotAllowed

Y probamos la libreria pdfminer sin llegar a nada claro